In [1]:
from selenium import webdriver
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
import statistics
import resources

## Scrapping the data
- Entering url and stracting data as a Pandas df
- After scrapping the data, df will be stored as CSV file
- If data has been already scrapped, data is stracted from CSV

In [2]:
# Check if exist a .csv with filename equals to TODAY DATETIME
if False:
#     NON_DESIRED_TEAMS = ['Team Durant','Team LeBron']
    # Getting data
    df = resources.data_scraper.scrape()
    # Filtering data
#     df = df[df[df.columns[0]] != (NON_DESIRED_TEAMS[0] or NON_DESIRED_TEAMS[1])]
#     df = df[df[df.columns[1]] != (NON_DESIRED_TEAMS[0] or NON_DESIRED_TEAMS[1])]
    # Casting columns
    df[df.columns[2]] = df[df.columns[2]].astype(np.int64)
    df[df.columns[3]] = df[df.columns[3]].astype(np.int64)
    # Adding total_score column
    df["TotalScore"] = df[df.columns[2]] + df[df.columns[3]]
    # Store CSV
    df.to_csv('data/11-04.csv', encoding='utf-8', index=False)
else:
    df = pd.read_csv('data/11-04.csv')

In [3]:
# NON_DESIRED_TEAMS = ['Team Durant','Team LeBron']
# df = df[df[df.columns[0]] != (NON_DESIRED_TEAMS[0] or NON_DESIRED_TEAMS[1])]
# df = df[df[df.columns[1]] != (NON_DESIRED_TEAMS[0] or NON_DESIRED_TEAMS[1])]
len(df) # 833 for today

833

#### Heatmap

In [5]:
# resources.graph.heatmap(df, f_size=10, cmap="afmhot_r",vmin=150, vmax=250)

### Frecuency histogram
- To know how many divisions or 'bins' should be done in the histogram, we must use Sturges law

$$ c = 1 + \dfrac{log(M)}{log(2)}$$

In [ ]:
# resources.graph.frecuency_histogram(df)


#### Boxplot

In [ ]:
# plt.boxplot(df['TotalScore'])
# plt.title("Total score distribution")

#### Normal dfistribution

In [ ]:
# resources.graph.normal_distribution(df, 'Atlanta Hawks')

## Data analysis
- Analysing data
- Store results in a new df

In [6]:
stats_df = resources.nba_stats.stats(df)

## Manually studying data

In [7]:
conversion_dict = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GS': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NO': 'New Orleans Pelicans',
    'NY': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHX': 'Phoenix Suns',
    'PHI': 'Philadelphia 76ers',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SA': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

In [8]:
future_games = [
    ['ATL', 'CHA'],
    ['BOS', 'DEN'],
    ['MIL', 'ORL'],
    ['NO', 'CLE'],
    ['CHI', 'MIN'],
    ['IND', 'MEM'],
    ['SA', 'DAL'],
    ['TOR', 'NY'],
    ['DET', 'LAC'],
    ['MIA', 'POR'],    
]

games_converted = list()
               
for game in future_games:
    home_team = conversion_dict[game[0]]
    away_team = conversion_dict[game[1]]
    game_ = [home_team, away_team]
    games_converted.append(game_)
    
for game in games_converted:
    df_ = stats_df.query("team == '{0}' or team == '{1}'".format(game[0], game[1]))
#     print(df_)
    print(df_.iloc[:,[0,1,4,5]])
    print("\n\n")
        

                 team        mean  factorDown   factorUp
10      Atlanta Hawks  225.350877    7.017544  10.526316
29  Charlotte Hornets  223.163636   10.909091   9.090909



              team        mean  factorDown  factorUp
8   Denver Nuggets  225.327273    5.454545  7.272727
12  Boston Celtics  222.600000   10.909091  7.272727



               team        mean  factorDown   factorUp
9   Milwaukee Bucks  232.218182    1.818182  21.818182
14    Orlando Magic  217.357143   12.500000   1.785714



                    team        mean  factorDown   factorUp
6    Cleveland Cavaliers  214.589286   25.000000   7.142857
11  New Orleans Pelicans  229.481481    9.259259  20.370370



                      team        mean  factorDown   factorUp
21  Minnesota Timberwolves  228.196429    5.357143  16.071429
24           Chicago Bulls  225.472727    3.636364  16.363636



                 team        mean  factorDown   factorUp
20     Indiana Pacers  226.129630    9.259259  16.666667
27  Memphi

## Training model

In [ ]:
def normalize(x, min_, max_):
    x = float(x)
    if x >= max_:
        return 1
    elif x <= min_:
        return 0
    return (x - min_) / (float(max_) - min_)

def denormalize(z, min_, max_):
    return z * (max_ - min_) + min_
    

def prepare_data(df):
    down, mean, up = df['factorDown'], df['mean'], df['factorUp']
    mean = normalize(mean, 165, 250)
    down = float(down)/100
    up = float(up)/100
    return down, mean, up
    

for index, row in df.iterrows():
    home, away, score = row['HomeTeam'], row['AwayTeam'], row['TotalScore']
    df_h = stats_df.query("team == '{0}'".format(home))
    df_a = stats_df.query("team == '{0}'".format(away))

    h_down, h_mean, h_up = prepare_data(df_h)
    a_down, a_mean, a_up = prepare_data(df_a)
    
    # Input for ML model
    X = np.array([h_down, h_mean, h_up, a_up, a_mean, a_up])
    # Expected result
    y = normalize(score, 165, 250)   
    
    print('Training model with input:\nX: {}\nExpected output:\ny: {}\n\n'.format(X, y))
    
    if index>=5:
        break